In [2]:
df = spark.read.csv("gs://dataproc-staging-us-central1-914709071200-uawfkl3d/2019-01-h1.csv", header=True, inferSchema=True)

new_dataset = df.select("passenger_count", "PULocationID", "DOLocationID", "total_amount")

new_dataset.select("passenger_count", "PULocationID", "DOLocationID", "total_amount").show(10)


+---------------+------------+------------+------------+
|passenger_count|PULocationID|DOLocationID|total_amount|
+---------------+------------+------------+------------+
|            1.0|       151.0|       239.0|        9.95|
|            1.0|       239.0|       246.0|        16.3|
|            3.0|       236.0|       236.0|         5.8|
|            5.0|       193.0|       193.0|        7.55|
|            5.0|       193.0|       193.0|       55.55|
|            5.0|       193.0|       193.0|       13.31|
|            5.0|       193.0|       193.0|       55.55|
|            1.0|       163.0|       229.0|        9.05|
|            1.0|       229.0|         7.0|        18.5|
|            2.0|       141.0|       234.0|        13.0|
+---------------+------------+------------+------------+
only showing top 10 rows



In [5]:
#split into train and test
trainDF, testDF = new_dataset.randomSplit([0.8, 0.2], seed=42)

# print number of rows
print(f"""There are {trainDF.count()} rows in the training set,
and {testDF.count()} rows in the test set""")


There are 2920930 rows in the training set,
and 730150 rows in the test set


In [9]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

# assembler
assembler = VectorAssembler(
    inputCols=["passenger_count", "PULocationID", "DOLocationID"],
    outputCol="features"
)
# decision Tree Regressor
dtr = DecisionTreeRegressor(
    featuresCol="features",
    labelCol="total_amount"
).setMaxBins(100)

# Make a pipeline
pipeline = Pipeline(stages=[assembler, dtr])

# Train model
model = pipeline.fit(trainDF)

predictions = model.transform(testDF)
predictions.select(
    "passenger_count", "PULocationID", "DOLocationID",
    "total_amount", "prediction"
).show(10)

# regression evaluator
evaluator = RegressionEvaluator(
    labelCol="total_amount",
    predictionCol="prediction",
    metricName="rmse"
)
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")


+---------------+------------+------------+------------+------------------+
|passenger_count|PULocationID|DOLocationID|total_amount|        prediction|
+---------------+------------+------------+------------+------------------+
|            0.0|         4.0|         4.0|         4.3| 17.06203337969388|
|            0.0|         4.0|        33.0|       17.75| 17.06203337969388|
|            0.0|         4.0|        68.0|        15.8|17.278418981559167|
|            0.0|         4.0|        79.0|        9.75|17.278418981559167|
|            0.0|         4.0|       125.0|         9.3|17.278418981559167|
|            0.0|         4.0|       170.0|       11.15|17.278418981559167|
|            0.0|         7.0|         7.0|        0.31| 17.06203337969388|
|            0.0|         7.0|         7.0|         6.3| 17.06203337969388|
|            0.0|         7.0|       112.0|        16.8|17.278418981559167|
|            0.0|         7.0|       138.0|        10.8|17.278418981559167|
+-----------

Root Mean Squared Error (RMSE): 24.548998093737836
